In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./results/final/pipeline/sampled_posts-mistral-123b-k_50.csv')
df_mapping = pd.read_csv('./datasets/multiclaim/fact_check_post_mapping.csv')
df['fact_check_ids'] = df['fact_check_ids'].apply(eval)
df['fact_check_claims'] = df['fact_check_claims'].apply(eval)
temp_df = pd.DataFrame()

for index, row in df.iterrows():
    post_id = row['post_id']
    fact_check_ids = row['fact_check_ids']
    fact_cehck_claims = row['fact_check_claims']
    post_text = row['post']
    for claim, fact_check_id in zip(fact_cehck_claims, fact_check_ids):
        temp_df = pd.concat([temp_df, pd.DataFrame({
            'post_id': [post_id],
            'fact_check_text': [claim],
            'fact_check_id': [fact_check_id],
            'post_text': [post_text]
        })])
        
temp_df['rating'] = ''
temp_df = temp_df.reset_index(drop=True)

for index, row in temp_df.iterrows():
    post_id = row['post_id']
    fact_check_id = row['fact_check_id']
    found = df_mapping[(df_mapping['fact_check_id'] == fact_check_id) & ((df_mapping['post_id'] == post_id))]
    if found.shape[0] > 0:
        temp_df.at[index, 'rating'] = 'Yes'
    else:
        temp_df.at[index, 'rating'] = 'No'

In [ ]:
temp_df.to_csv('./baseline.csv', index=False)